In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

### Extract CSVs into DataFrames

In [16]:
happiness_file = "resources/data/happiness/2017_Happiness.csv"
happiness_df = pd.read_csv(happiness_file)
happiness_df.head()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [8]:
cc_file = "resources/data/happiness/country_codes.csv"
country_code_df = pd.read_csv(cc_file)
country_code_df.head()

,COUNTRY,A2 (ISO),A3 (UN),NUM (UN),DIALING CODE
0,Afghanistan,AF,AFG,4,93
1,Albania,AL,ALB,8,355
2,Algeria,DZ,DZA,12,213
3,American Samoa,AS,ASM,16,1-684
4,Andorra,AD,AND,20,376


In [14]:
json_file = "resources/data/happiness/convertcsv.json"
country_facts_df = pd.read_json(json_file)
country_facts_df.head()


,Agriculture,Arable (%),Area (sq. mi.),Birthrate,Climate,Coastline (coast,Country,Crops (%),Deathrate,GDP ($ per capita),Industry,Infant mortality (per 1000 births),Literacy (%),Net migration,Other (%),Phones (per 1000),Pop. Density (per sq. mi.),Population,Region,Service
0,0.380,12.13,647500,46.60,1.0,{'area ratio)': 0},Afghanistan,0.22,20.34,700.0,0.240,163.07,36.0,23.06,87.65,3.2,48.0,31056997,ASIA (EX. NEAR EAST),0.380
1,0.232,21.09,28748,15.11,3.0,{'area ratio)': 1.26},Albania,4.42,5.22,4500.0,0.188,21.52,86.5,-4.93,74.49,71.2,124.6,3581655,EASTERN EUROPE,0.579
2,0.101,3.22,2381740,17.14,1.0,{'area ratio)': 0.04},Algeria,0.25,4.61,6000.0,0.600,31.00,70.0,-0.39,96.53,78.1,13.8,32930091,NORTHERN AFRICA,0.298
3,NaN,10.00,199,22.46,2.0,{'area ratio)': 58.29},American Samoa,15.00,3.27,8000.0,NaN,9.27,97.0,-20.71,75.00,259.5,290.4,57794,OCEANIA,NaN
4,NaN,2.22,468,8.71,3.0,{'area ratio)': 0},Andorra,0.00,6.25,19000.0,NaN,4.05,100.0,6.60,97.78,497.2,152.1,71201,WESTERN EUROPE,NaN


### Transform World Happiness DataFrame

In [44]:
# Create a filtered dataframe from specific columns
happiness_cols = ["Country", "Happiness.Score", "Economy..GDP.per.Capita.","Family","Health..Life.Expectancy.","Freedom","Generosity","Trust..Government.Corruption.","Dystopia.Residual"]
happiness_transformed= happiness_df[happiness_cols].copy()

# Rename the column headers
happiness_transformed = happiness_transformed.rename(columns={"Happiness.Score": "Happiness_Score",
                                                          "Economy..GDP.per.Capita.": "GDP_Per_Capita_Score",
                                                          "Family": "Family_Score",
                                                          "Health..Life.Expectancy.":"Health_Score",
                                                           "Freedom":"Freedom_Score",
                                                           "Generosity":"Generosity_Score",
                                                           "Trust..Government.Corruption.": "Trust_Govt_Score",
                                                           "Dystopia.Residual":"Dystopia_Score"})

# Clean the data by dropping duplicates and setting the index
happiness_transformed.drop_duplicates("Country", inplace=True)

happiness_transformed.head()

,Country,Happiness_Score,GDP_Per_Capita_Score,Family_Score,Health_Score,Freedom_Score,Generosity_Score,Trust_Govt_Score,Dystopia_Score
0,Norway,7.537,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,7.522,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,7.504,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,7.494,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,7.469,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


### Transform Country Facts DataFrame

In [33]:
facts_cols = ["Country", "Region", "Population", "Pop. Density (per sq. mi.)","Area (sq. mi.)","Birthrate","Deathrate"]
country_facts_transformed = country_facts_df[facts_cols].copy()

# Rename the column headers
country_facts_transformed = country_facts_transformed.rename(columns={"Pop. Density (per sq. mi.)": "pop_density",
                                                         "County Name (Licensee)": "county_name",
                                                         "Area (sq. mi.)": "Area_Sq_Miles"})

# Set index

country_facts_transformed.head()

,Country,Region,Population,pop_density,Area_Sq_Miles,Birthrate,Deathrate
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,48.0,647500,46.60,20.34
1,Albania,EASTERN EUROPE,3581655,124.6,28748,15.11,5.22
2,Algeria,NORTHERN AFRICA,32930091,13.8,2381740,17.14,4.61
3,American Samoa,OCEANIA,57794,290.4,199,22.46,3.27
4,Andorra,WESTERN EUROPE,71201,152.1,468,8.71,6.25


### Create database connection

In [27]:
connection_string = "root:Coronad01!@localhost/"
engine = create_engine(f'mysql://{connection_string}')

connection = engine.connect()

In [28]:
connection.execute('create schema happiness_db; ')

In [29]:
connection.happiness_db

AttributeError: 'Connection' object has no attribute 'happiness_db'

In [30]:
db_connection_string = "root:Coronad01!@localhost/happiness_db"
db_engine = create_engine(f'mysql://{db_connection_string}')

# Confirm tables
db_engine.table_names()

[]

### Load DataFrames into database

In [50]:
happiness_transformed.to_sql(name='happiness', con=db_engine, if_exists='append', index=True)

InternalError: (pymysql.err.InternalError) (1054, "Unknown column 'index' in 'field list'")
[SQL: INSERT INTO happiness (`index`, `Country`, `Happiness_Score`, `GDP_Per_Capita_Score`, `Family_Score`, `Health_Score`, `Freedom_Score`, `Generosity_Score`, `Trust_Govt_Score`, `Dystopia_Score`) VALUES (%(index)s, %(Country)s, %(Happiness_Score)s, %(GDP_Per_Capita_Score)s, %(Family_Score)s, %(Health_Score)s, %(Freedom_Score)s, %(Generosity_Score)s, %(Trust_Govt_Score)s, %(Dystopia_Score)s)]
[parameters: ({'index': 0, 'Country': 'Norway', 'Happiness_Score': 7.537000179290769, 'GDP_Per_Capita_Score': 1.6164631843566901, 'Family_Score': 1.53352355957031, 'Health_Score': 0.796666502952576, 'Freedom_Score': 0.635422587394714, 'Generosity_Score': 0.36201223731041, 'Trust_Govt_Score': 0.31596383452415505, 'Dystopia_Score': 2.27702665328979}, {'index': 1, 'Country': 'Denmark', 'Happiness_Score': 7.52199983596802, 'GDP_Per_Capita_Score': 1.48238301277161, 'Family_Score': 1.55112159252167, 'Health_Sco

In [36]:
country_facts_transformed.to_sql(name='country_facts', con=db_engine, if_exists='append', index=True)

In [41]:
db_engine.table_names()

['country_facts', 'happiness']

In [48]:
results = db_engine.execute('select * from country_facts')

In [49]:
for item in results:
    print(item)

(0, 'Afghanistan ', 'ASIA (EX. NEAR EAST)         ', 31056997, 48.0, 647500, 46.6, 20.34)
(1, 'Albania ', 'EASTERN EUROPE                     ', 3581655, 124.6, 28748, 15.11, 5.22)
(2, 'Algeria ', 'NORTHERN AFRICA                    ', 32930091, 13.8, 2381740, 17.14, 4.61)
(3, 'American Samoa ', 'OCEANIA                            ', 57794, 290.4, 199, 22.46, 3.27)
(4, 'Andorra ', 'WESTERN EUROPE                     ', 71201, 152.1, 468, 8.71, 6.25)
(5, 'Angola ', 'SUB-SAHARAN AFRICA                 ', 12127071, 9.7, 1246700, 45.11, 24.2)
(6, 'Anguilla ', 'LATIN AMER. & CARIB    ', 13477, 132.1, 102, 14.17, 5.34)
(7, 'Antigua & Barbuda ', 'LATIN AMER. & CARIB    ', 69108, 156.0, 443, 16.93, 5.37)
(8, 'Argentina ', 'LATIN AMER. & CARIB    ', 39921833, 14.4, 2766890, 16.73, 7.55)
(9, 'Armenia ', 'C.W. OF IND. STATES ', 2976372, 99.9, 29800, 12.07, 8.23)
(10, 'Aruba ', 'LATIN AMER. & CARIB    ', 71891, 372.5, 193, 11.03, 6.68)
(11, 'Australia ', 'OCEANIA                            ', 2026